# Polynomial handling
Monomials (here as an example in 3 variables) are handled as follows:
* monomials of degree 1 are coded as [1, 0, 0] for $x_1$, [0, 1, 0] for $x_2$ and [0, 0, 1] for $x_3$
* for degree $\ge 2$ monomials (here = 2 example) we save them as [2, 0, 0] for $x_1^2$, [1, 1, 0] for $x_1 x_2$, etc. 

Should be extendable to SparseArrays without much problem. For ease of understanding and testing I used normal Arrays/Vectors for now.

Also most methods still need to be given their expected input type, e.g. Vector{Int}, etc.

In [1]:
function unique_indices(x)
    
    #=
    This function returns the indices of unique elements in x.
    "i -> x[i], 1:length(x)" means we want to return the value i if and 
    only if x[i] is unique for i in 1 through length(x)
    =#
    
    a = unique(i -> x[i], 1:length(x))
        
    return a
    
end

unique_indices (generic function with 1 method)

In [2]:
function monomial_multiplication(m1::Vector{Int}, m2::Vector{Int})
    
    #=
    This function multiplies two monomials m1 and m2 by element-wise addition of their exponents.
    The monomials are encoded as arrays where entry i is the exponent of variable x_i.
    =#
    
    result = similar(m1)  # creates output vector of same dimension as input vectors
    
    for i in 1:length(m1)
        result[i] = m1[i] + m2[i]
    end
    
    return result
    
end

monomial_multiplication (generic function with 1 method)

In [3]:
# Example of monomial_multiplication
m1 = [2, 3, 0]  # represents x_1^2 * x_2^3
m2 = [1, 0, 2]  # represents x_1^1 * x_3^2
new_m = monomial_multiplication(m1, m2)  # should construct the monomial x_1^3 * x_2^3 * x_3^2
new_m == [3, 3, 2]  # check for correct result

true

In [4]:
function monomial_evaluation(m::Vector, x::Vector, coeff::Real=1)
    
    #=
    This function evaluates a monomial m at the point x.
    The optional "coeff" argument can be input to the function to use a different coeff than 1.
    For-loop seems pretty fast compared to naive alternatives using broadcasting.
    =#
    
    result = 1
    
    for i in 1:length(m)
        result *= x[i]^m[i]
    end
    
    return coeff * result
    
end

monomial_evaluation (generic function with 2 methods)

In [5]:
# Example of monomial_evaluation
m = [3, 3, 2]  # monomial x_1^3 * x_2^3 * x_3^2
x = [1, 2, 3]  # point [x_1, x_2, x_3] at which we want to evaluate m
m_eval = monomial_evaluation(m, x)  # should result in 1^3 * 2^3 * 3^2 = 1 * 8 * 9 = 72
m_eval == 72

true

In [6]:
# Example of monomial_evaluation with "coeff" changed
m = [3, 3, 2]  # monomial x_1^3 * x_2^3 * x_3^2
x = [1, 2, 3]  # point [x_1, x_2, x_3] at which we want to evaluate m
m_eval = monomial_evaluation(m, x, 3)  # should result in 3 * 1^3 * 2^3 * 3^2 = 3 * 1 * 8 * 9 = 3 * 72
m_eval == 3 * 72

true

In [7]:
function set_evaluation(m, X, coeff::Any=1)
    
    #=
    This function evaluates a set of points X under a given monomial m 
    and returns the output as a vector of lenght |X|.
    =#
    
    results = ones(length(X))
        
    for i in 1:length(X)
        results[i] = monomial_evaluation(m, X[i])
    end
    
    return coeff * results
    
end

set_evaluation (generic function with 2 methods)

In [8]:
# Example of set_evaluation
X = [[1, 2, 3], [1, 2, 4], [3, 2, 1]]  # set of points to be evaluated
m = [2, 2, 2]  # monomial under which to evaluate
y = set_evaluation(m, X)
y == [4*9, 4*16, 9*4]  # check for correct result

true

In [9]:
function set_eval_monomial_set(M, X, coeffs::Any=1)
    
    #=
    This function evaluates a set of points X under a set of monomials M and
    returns the output as a vector of vectors of size |M| x |X|.
    The i-th entry of the output is the evaluation vector of the set X under the i-th monomial in M in sequence.
    The argument "coeffs" can optionally add coefficients to be used for each monomial
    =#
    
    result = [[] for _ in 1:length(M)]
    
    for i in 1:length(M)
        result[i] = set_evaluation(M[i], X)
    end
    
    return coeffs .* result
    
end

set_eval_monomial_set (generic function with 2 methods)

In [10]:
# Example of set_eval_monomial_set
M = [[2, 2, 2], [2, 1, 3]]  # set of monomials under which to evaluate
y = set_eval_monomial_set(M, X)
y == [
    [4*9, 4*16, 9*4],
    [2*27, 2*4^3, 9*2]
]  # check for correct result

true

We probably don't need all these methods and they are certainly not perfectly optimized but for now they do what they're supposed to do. 

These basic Julia implementations still are faster than any basic broadcasting implementations I tried to reduce code length. For-loops are indeed very fast.